# Sub-seasonal range re-forecasts example

## Objective  
This notebook will show you:
- how to find which dates and steps to use to download the re-forecacsts - weekly steps from sub-seasonal re-forecasts
- download the re-forecasts
- calculate the mean of the fields for each step
- calculate the percentiles for each step

Please note that the climate built using this notebook is only valid for the date it was built. If you want to build the climate on some other day, you need to download the data again with the correct steps.

In [1]:
import datetime
from earthkit.time import Sequence, model_climate_dates, date_range, WeeklySequence
import metview as mv
import requests
import os

Sequenece is Abstract representation of a sequence of dates.  
For the re-forecasts we use either ecmwf-4days or ecmwf-2days sequence. These are built in, and represent configuration of ECMWF forecast systems.

First example will be sub-seasonal range forecast.

In [2]:
sequence = Sequence.from_resource("ecmwf-2days")
sequence

MonthlySequence(days=[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31], excludes={(2, 29)})

Next thing we need is to know what dates we need to build the model climate for today's forecast.  
Form medium range model climate (M climate) we can get this using the model_climate_dates function.

model_climate_dates(reference: date, start: date | int, end: date | int, before: timedelta | int, after: timedelta | int, sequence: Sequence)→ Iterator[date]  
Parameters:
- `reference (datetime.date)` – Reference date for the climate
- `start (datetime.date or int)` – Start of the climatological period. Either a full date or a year
- `end (datetime.date or int)` – End of the climatological period. Either a full date or a year
- `before (datetime.timedelta or int)` – Cut-off before the reference date. Either a timedelta or a number of days
- `after (datetime.timedelta or int)` – Cut-off after the reference date. Either a timedelta or a number of days
- `sequence (earthkit.time.sequence.Sequence)` – Sequence of available dates in the reference set

Returns: Sequence of dates

Now we have a sequence, we can find a closest reforecast date for today, and from there, all the reforecast dates we need to calculate the model climate.

In [3]:
today = datetime.date.today()
today

datetime.date(2024, 10, 9)

Now we need to calculate the closest day of the ECMWF re-forecasts.  
We will use the **nearest** funciton.

In [4]:
clim_date = sequence.nearest(today)
clim_date

datetime.date(2024, 10, 9)

Next we will calculate the sequence of 9 dates, around our climatology date (including the climatology date).  
For this we can use the bracket function.  
We need to give it `clim_date`, number of days around `clim_date` we want reforecasts for and we need to set `strict` to False to include the `clim_date` in the set of dates.

In [5]:
clim_dates = sequence.bracket(clim_date,2,strict=False)
clim_dates

<generator object Sequence.bracket at 0x10d945be0>

We can loop through the `clim_dates` to see what we got:

In [6]:
for c in clim_dates:
    print(c)

2024-10-05
2024-10-07
2024-10-09
2024-10-11
2024-10-13


The steps we need from the reforecast will depend on the **day of the week of the forecast**.  
This is because at ECMWF we are creating weekly climatologies to calculate the forecast anomalies, for example.

Every **forecast week starts on Monday**. Therefore, depending on the day of the week of the forecast, we will need different steps from the re-forecasts.  

The steps are defined in this table:

| DOW | steps |
| --- | --- |
| Monday | 0-168/168-336/336-504/504-672/672-840/840-1008 |
| Tuesday | 144-312/312-480/480-648/648-816/816-984 |
| Wednesday | 120-288/288-456/456-624/624-792/792-960 |
| Thursday | 96-264/264-432/432-600/600-768/768-936/936-1104 |
| Friday | 72-240/240-408/408-576/576-744/744-912/912-1080 |
| Saturday | 48-216/216-384/384-552/552-720/720-888/888-1056 |
| Sunday | 24-192/192-360/360-528/528-696/696-864/864-1032 |

We can, of course do something like: 
```python
    if dow == 0:
        steps = "0-168/168-336/336-504/504-672/672-840/840-1008"
    ...
    elif dow == 6:
        steps = "24-192/192-360/360-528/528-696/696-864/864-1032"
```

but we can make it simpler.  

First we find out the day of the week of the date of the climatology.  

Note that the days of the week in python start with 0, so 1 is Tuesday.

In [7]:
today.weekday()

2

We can use the following formula to find out the first step:
```python
first_step = ((7 - dow) % 7)*24
```
And then add weekly steps (168 hours) to build all the steps we need.

In [8]:
dow = today.weekday()
first_step = ((7 - dow) % 7)*24

first_step_end = first_step + 168
print(first_step_end)

288


We can check if this is correct result for all the days in the week by running next cell:

In [9]:
for dow in range(7):
    first_step = ((7 - dow) % 7)*24
    print(first_step)

0
144
120
96
72
48
24


Next step is to create the list of all the steps for a climatology date.  
This code is not used in this notebook, but it is helpful if you want to get the re-forecast data from MARS.

In [10]:
dow = today.weekday()

first_step = ((7 - dow) % 7)*24
maxstep = 937 #this is maximum of all the left ranges in the re-forecast
weekly_step = 168
the_steps = ""

for s in range(first_step, maxstep, weekly_step):
    step_string = str(s) + '-' + str(s+168)
    if s + weekly_step < maxstep:
        step_string += '/'
    the_steps += step_string

print(the_steps)

120-288/288-456/456-624/624-792/792-960


We have prepared a test set of reforecasts available at the address: https://xdiss.ecmwf.int/ecpds/home/rcp/  
From here users can download medium and sub-seasonal range re-forecasts to test their system.  
**The data here is available strictly for testing and the operational and commercial use is not allowed.**

Now we can put it all together.

In the next cell we will download the files from the RCP destination needed **for re-forecasts that are matching today's forecast**.  

1. We loop through all re-forecast dates (clim_dates)
2. And for each date in clim_dates we download the file containing the appropriate weekly steps.
3. From those files we select only 2 metre temperature, save this data and delete the original file.
4. If the file is already there, it will be skipped.

In [ ]:
today = datetime.date.today()
dow = today.weekday()

clim_date = sequence.nearest(today)
clim_dates = sequence.bracket(clim_date,2,strict=False)

maxstep = 937
first_step = ((7 - dow) % 7)*24
weekly_step = 168

print(first_step)

for date in clim_dates:
    month, day = date.month, date.day
    print(month, day)

    ref_date = date.strftime("%Y%m%d")
    
    month_day_str = date.strftime("%m%d")
    fdate = date + datetime.timedelta(days=(first_step_end/24))
    print(fdate)
    
    for s in range(first_step, maxstep, weekly_step):
        print(s + weekly_step)
        if s < maxstep:
            month_day_str_f = fdate.strftime("%m%d")
            url = f'https://xdiss.ecmwf.int/ecpds/home/rcp/{ref_date}/subseasonal/B4H{month_day_str}0000{month_day_str_f}____0079.RCP90DG1000001' 
            print(url)
            local_filename = f"B4H{month_day_str}0000{month_day_str_f}____0079.RCP90DG1000001"
            t2m_filename = f't2m_{local_filename}.grib'
            
            if os.path.exists(local_filename) or os.path.exists(t2m_filename):
                pass
            else:
                # Make the request to download the file
                response = requests.get(url)
                
                # Check if the request was successful
                if response.status_code == 200:
                    # Write the content to the local file
                    with open(local_filename, 'wb') as f:
                        f.write(response.content)
                    print(f"File downloaded successfully and saved as {local_filename}")
                else:
                    print(f"Failed to download file. Status code: {response.status_code}")
    
    
            if os.path.exists(t2m_filename):
                print(f"File {t2m_filename} is already there.")
            else:
                data = mv.Fieldset(path=local_filename)
            
                t2m = data.select(shortName='2t')
                
                t2m.write(t2m_filename)
                print(f"Saved {t2m_filename}")
        
                if os.path.exists(local_filename):
                  os.remove(local_filename)
                else:
                  print("The file does not exist")

            fdate = fdate + datetime.timedelta(days=7)

Let's explore the files we have just downloaded

In [12]:
data = mv.Fieldset(path="t2m_B4H*.grib")
data.describe()

parameter,typeOfLevel,level,date,time,step,number,paramId,class,stream,type,experimentVersionNumber
2t,surface,0,"20241005,20241007,...",0,"288,456,...","0,1,...",167,od,eefh,fcmean,0079


In [13]:
data.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
Message,,,,,,,,,,
0,ecmf,2t,surface,0,20151005,0,120-288,fcmean,0,regular_ll
1,ecmf,2t,surface,0,20231005,0,120-288,fcmean,0,regular_ll
2,ecmf,2t,surface,0,20161005,0,120-288,fcmean,0,regular_ll
3,ecmf,2t,surface,0,20041005,0,120-288,fcmean,0,regular_ll
4,ecmf,2t,surface,0,20081005,0,120-288,fcmean,0,regular_ll
...,...,...,...,...,...,...,...,...,...,...
3295,ecmf,2t,surface,0,20101009,0,792-960,fcmean,1,regular_ll
3296,ecmf,2t,surface,0,20151009,0,792-960,fcmean,8,regular_ll
3297,ecmf,2t,surface,0,20101009,0,792-960,fcmean,5,regular_ll


We can see that for Tuesday and Wednesday, we have 5500 fields: 5 steps x 11 ensemble members (number 0-10) x 5 dates x 20 years.  
All the other days have 6 weekly steps so we have 6600 fields: 6 steps x 11 ensemble members (number 0-10) x 5 dates x 20 years.

Note: When step is in the form of range, we need to use the parameter **stepRange**.

Now we can calculate the mean value of all the emsemble members over all 20 years of the reforecasts for one step.  
To get one step, first we need to convert our string with steps to the list of steps and take the first element of the list.

In [14]:
steps = the_steps.split("/")
steps

['120-288', '288-456', '456-624', '624-792', '792-960']

In [15]:
first_step = steps[0]

In [16]:
one_step = data.select(stepRange = first_step)
one_step.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
Message,,,,,,,,,,
0,ecmf,2t,surface,0,20151005,0,120-288,fcmean,0,regular_ll
1,ecmf,2t,surface,0,20231005,0,120-288,fcmean,0,regular_ll
2,ecmf,2t,surface,0,20161005,0,120-288,fcmean,0,regular_ll
3,ecmf,2t,surface,0,20041005,0,120-288,fcmean,0,regular_ll
4,ecmf,2t,surface,0,20081005,0,120-288,fcmean,0,regular_ll
...,...,...,...,...,...,...,...,...,...,...
655,ecmf,2t,surface,0,20211009,0,120-288,fcmean,7,regular_ll
656,ecmf,2t,surface,0,20111009,0,120-288,fcmean,9,regular_ll
657,ecmf,2t,surface,0,20091009,0,120-288,fcmean,1,regular_ll


Now we simply calculate the mean over this one step

In [17]:
one_step_mean = mv.mean(one_step)
one_step_mean.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,gridType
Message,,,,,,,,,
0,ecmf,2t,surface,0,20151005,0,120-288,fcmean,regular_ll


We can plot the data to quickly check the result.  
Begin with adding some automatic styling and zoom into the area.

In [18]:
data_area = [50,10,40,30]
margins = [2, -2, -2, 2]
view_area = [a + b for a, b in zip(data_area, margins)]

In [19]:
coastlines = mv.mcoast()
view = mv.geoview(map_area_definition="corners", area=view_area, coastlines=coastlines)
cont_auto = mv.mcont(legend=True, contour_automatic_setting="ecmwf", grib_scaling_of_derived_fields=True)

In [20]:
mv.plot(view, one_step_mean, cont_auto)

Image(value=b'', layout="Layout(visibility='hidden')")

Label(value='Generating plots....')

Now let's finally calculate mean for each step.  
We can do this by calculating mean over **number** and **date** dimension.

Please note that when doing the calculations, Metview will **keep the metadata of the first field**.

In [21]:
mean_t2m = data.mean(dim=["number", "date"],
    preserve_dims=["shortName", "level", "stepRange", "time"])

In [22]:
mean_t2m.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,gridType
Message,,,,,,,,,
0,ecmf,2t,surface,0,20151005,0,120-288,fcmean,regular_ll
1,ecmf,2t,surface,0,20151005,0,288-456,fcmean,regular_ll
2,ecmf,2t,surface,0,20151005,0,456-624,fcmean,regular_ll
3,ecmf,2t,surface,0,20151005,0,624-792,fcmean,regular_ll
4,ecmf,2t,surface,0,20151005,0,792-960,fcmean,regular_ll


In [23]:
mean_t2m.describe()

parameter,typeOfLevel,level,date,time,step,paramId,class,stream,type,experimentVersionNumber
2t,surface,0,20241005,0,"288,456,...",167,od,eefh,fcmean,0079


In [24]:
mv.plot(view, mean_t2m, cont_auto)

Image(value=b'', layout="Layout(visibility='hidden')")

Label(value='Generating plots....')

Please note that if you don't see the plot above, you need to install the **ipywidgets** into the Python environment you are currently using. We are not importing the ipywidgets directly, but Metview is using it internally.

## Compute percentiles
Last thing left to do is to compute the percentiles. 
Here we compute the percentiles for the first step range.

In [25]:
percentiles = list(range(101))
pc = mv.percentile(data=one_step, percentiles=percentiles)
pc.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
Message,,,,,,,,,,
0,ecmf,2t,surface,0,20151005,0,120-288,fcmean,0,regular_ll
1,ecmf,2t,surface,0,20231005,0,120-288,fcmean,0,regular_ll
2,ecmf,2t,surface,0,20161005,0,120-288,fcmean,0,regular_ll
3,ecmf,2t,surface,0,20041005,0,120-288,fcmean,0,regular_ll
4,ecmf,2t,surface,0,20081005,0,120-288,fcmean,0,regular_ll
5,ecmf,2t,surface,0,20211005,0,120-288,fcmean,0,regular_ll
6,ecmf,2t,surface,0,20191005,0,120-288,fcmean,0,regular_ll
7,ecmf,2t,surface,0,20051005,0,120-288,fcmean,0,regular_ll
8,ecmf,2t,surface,0,20201005,0,120-288,fcmean,0,regular_ll


In [26]:
mv.plot(view, pc, cont_auto)

Image(value=b'', layout="Layout(visibility='hidden')")

Label(value='Generating plots....')

You can loop over all the steps and calculate the percentiles as well.